# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [55]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, avg, col, concat, desc, explode, lit, min, max, split, udf, when, isnull, collect_list
from pyspark.sql.types import IntegerType, BooleanType, FloatType
from pyspark.ml.feature import VectorAssembler, StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
file_name = "mini_sparkify_event_data.json"
df = spark.read.json(file_name)
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [4]:
df.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [5]:
len(df.columns)

18

In [6]:
df.count()

286500

In [7]:
df.select([count(when(isnull(i), i)).alias(i) for i in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [8]:
df.select('auth').distinct().show()

+----------+
|      auth|
+----------+
|Logged Out|
| Cancelled|
|     Guest|
| Logged In|
+----------+



In [9]:
df.select('level').distinct().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



In [10]:
df.select('method').distinct().show()

+------+
|method|
+------+
|   PUT|
|   GET|
+------+



In [11]:
df.select('status').distinct().show()

+------+
|status|
+------+
|   307|
|   404|
|   200|
+------+



In [12]:
df.select('page').distinct().show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
| Submit Registration|
|            Settings|
|               Login|
|            Register|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
+--------------------+
only showing top 20 rows



### Select useful fileds

In [13]:
df_clean = df.select(
                    'artist',
                    'auth',
                    'gender',
                    'itemInSession',
                    'length',
                    'level',
                    'location',
                    'method',
                    'page',
                    'registration',
                    'sessionId',
                    'song',
                    'status',
                    'ts',
                    'userAgent',
                    'userId')

In [14]:
df_clean.where((col("auth") == "Cancelled")).select("userId").distinct().count()

52

In [15]:
df_clean.select("userId").distinct().count()

226

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

### Define Churn

In [16]:
df_churn = df_clean.groupby('userId').agg(collect_list('auth').alias("auths"))
churned = udf(lambda x: 'Cancelled' in x)
df_churn = df_churn.withColumn("Churned", churned(df_churn.auths))
df_churn = df_churn.drop('auths')
df_churn.show()

+------+-------+--------------------+---------+------+-------------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+
|userId|Churned|              artist|     auth|gender|itemInSession|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|
+------+-------+--------------------+---------+------+-------------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+
|100003|   true|     Black Eyed Peas|Logged In|     F|            0|193.88036| free|Miami-Fort Lauder...|   PUT|NextSong|1537309344000|        3|          Gone Going|   200|1539100420000|Mozilla/5.0 (Wind...|
|100003|   true|            Paramore|Logged In|     F|            1|193.56689| free|Miami-Fort Lauder...|   PUT|NextSong|1537309344000|        3|All We Know (Albu..

In [17]:
df_label = df_churn.join(df_clean,'userId')
df_label.groupby("Churned").avg("length").collect()
df_label.select(["userId","Churned"]).distinct().groupBy("Churned").count().collect()

+------+-------+
|userId|Churned|
+------+-------+
|100010|  false|
|200002|  false|
|   125|   true|
|   124|  false|
|    51|   true|
|     7|  false|
|    15|  false|
|    54|   true|
|   155|  false|
|100014|   true|
|   132|  false|
|   154|  false|
|   101|   true|
|    11|  false|
|   138|  false|
|300017|  false|
|100021|   true|
|    29|   true|
|    69|  false|
|   112|  false|
+------+-------+
only showing top 20 rows



In [21]:
songsPlayed = df_label.where(col('song')!='null').groupby("userId").agg(count(col('song')).alias('SongsPlayed')).orderBy('userId')
songsPlayed.show()

+------+-----------+
|userId|SongsPlayed|
+------+-----------+
|    10|        673|
|   100|       2682|
|100001|        133|
|100002|        195|
|100003|         51|
|100004|        942|
|100005|        154|
|100006|         26|
|100007|        423|
|100008|        772|
|100009|        518|
|100010|        275|
|100011|         11|
|100012|        476|
|100013|       1131|
|100014|        257|
|100015|        800|
|100016|        530|
|100017|         52|
|100018|       1002|
+------+-----------+
only showing top 20 rows



In [22]:
df_features = df_churn.join(songsPlayed,'userId')

In [ ]:
thumbsUp = df_label.where(df_label.page=='Thumbs Up').groupby("userId").agg(count(col('page')).alias('ThumbsUp')).orderBy('userId')
thumbsUp.show()

In [ ]:
thumbsDown = df_label.where(df_label.page=='Thumbs Down').groupby("userId").agg(count(col('page')).alias('ThumbsDown')).orderBy('userId')
thumbsDown.show()

In [ ]:
allThumbs = thumbsUp.join(thumbsDown,'userId')

In [ ]:
df_features = df_features.join(allThumbs,'userId')

In [23]:
days = df_label.groupby('userId').agg(((max(col('ts')) - min(col('ts')))/86400000).alias("Days"))

In [27]:
df_features = df_features.join(days,'userId')

In [30]:
SongAddedToPlaylist = df_label.where(col('page')!='Add to Playlist').groupby("userId").agg(count(col('page')).alias('SongAddedToPlaylist'))

In [31]:
df_features = df_features.join(SongAddedToPlaylist,'userId')

In [32]:
FreindAdded = df_label.where(col('page')!='Add Friend').groupby("userId").agg(count(col('page')).alias('FreindAdded'))

In [33]:
df_features = df_features.join(FreindAdded,'userId')

In [1]:
# df_features.show()

In [38]:
# Vectorize and scale features
assembler = VectorAssembler(inputCols=["SongsPlayed", "ThumbsUp", "ThumbsDown", "Days", "SongAddedToPlaylist", "FreindAdded"], 
                            outputCol="FeatureVector")
df_features = assembler.transform(df_features)

In [39]:
scaler = StandardScaler(inputCol="FeatureVector", outputCol="ScaledFeatures", withStd=True)
scalerModel = scaler.fit(df_features)
df_features = scalerModel.transform(df_features)

In [62]:
ConvertToInt = udf(lambda x: 1 if x=="true" else 0, IntegerType())
df_features = df_features.withColumn('label', ConvertToInt(df_features.Churned))

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [64]:
df_model = df_features.select(col('label'),col('ScaledFeatures').alias('features'))
df_model.take(1)

[Row(label=0, features=DenseVector([0.2464, 0.2554, 0.3786, 2.5603, 0.2857, 0.2851]))]

In [65]:
train, test = df_model.randomSplit([0.8, 0.2], seed=50)

In [66]:
logit = LogisticRegression(maxIter=10,regParam=0.0)
gbt = GBTClassifier(maxDepth=5,maxIter=10,seed=50)

In [67]:
# logistic
el = MulticlassClassificationEvaluator(metricName='f1')
paramGrid = ParamGridBuilder() \
    .addGrid(logit.regParam,[0.01, 0.05, 0.1]) \
    .build()
crossval = CrossValidator(estimator=logit,
                          estimatorParamMaps=paramGrid,
                          evaluator=el,
                          numFolds=3)

In [69]:
cvModel_lr = crossval.fit(train)
#report the f1 score 
cvModel_lr.avgMetrics

[0.8518888689299173, 0.8352876592586012, 0.8142416643159942]

In [70]:
#gradient boosted tree models
e1 = MulticlassClassificationEvaluator(metricName='f1')
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth,[2, 5, 10]) \
    .build()
crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=e1,
                          numFolds=3)

In [71]:
cvModel_boost = crossval.fit(train)
#report the f1 score 
cvModel_boost.avgMetrics

[0.853390824984672, 0.8054334046770071, 0.8104963073752878]

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.